# CS1671 Machine Comprehension Project
### Jacob Emmerson
11/30/23

In [1]:
import pandas as pd
import numpy as np
import re # regular expression
import seaborn as sns
import matplotlib.pyplot as plt

from THM_class import *
from RTE_class import *
from dep import *

In [2]:
train_stories = load_stories('data/mc500.train.tsv')
train_answers = load_answers('data/mc500.train.ans')
train_stories.T.head(2)

,story_id,story,q1,q2,q3,q4,q1_a,q2_a,q3_a,q4_a
0,mc500.train.0,Alyssa got to the beach after a long trip. She...,one: What city is Alyssa in?,one: Why did Alyssa go to Miami?,multiple: How many friends does Alyssa have?,multiple: What did Alyssa eat at the restaurant?,"{'A': 'trip', 'B': 'Miami', 'C': 'Atlanta', 'D...","{'A': 'swim', 'B': 'travel', 'C': 'visit frien...","{'A': '1', 'B': '2', 'C': '3', 'D': '4'}","{'A': 'steak', 'B': 'soup', 'C': 'salad', 'D':..."
1,mc500.train.1,"One morning, Elena woke up, much like she did ...",multiple: What is the very first thing Elena d...,one: At what time of day does this story take ...,multiple: What happened to Mr. Fish in the end?,multiple: What was missing from the fish bowl?,"{'A': 'she says hello to the tree', 'B': 'she ...","{'A': 'Before the tree', 'B': 'At the end of t...","{'A': 'He got put back in the bowl, but he was...","{'A': 'the rocks', 'B': 'Mr. Fish', 'C': 'the ..."


In [3]:
dev_stories = load_stories('data/mc500.dev.tsv')
dev_answers = load_answers('data/mc500.dev.ans')
dev_stories.T.head(2)

,story_id,story,q1,q2,q3,q4,q1_a,q2_a,q3_a,q4_a
0,mc500.dev.0,"One sunny morning, Stewart chose to go to the ...","multiple: If the rain arrived on Saturday, wha...",one: What was Stewart doing at the beach?,one: What did the bug want from Stewart?,multiple: What was the bug that Stewart found ...,"{'A': 'Friday', 'B': 'Monday', 'C': 'Sunday', ...","{'A': 'reading a newspaper', 'B': 'looking for...","{'A': 'food', 'B': 'to read Stewart's newspape...","{'A': 'beetle', 'B': 'caterpillar', 'C': 'ant'..."
1,mc500.dev.1,My mommy has a really cool job! She makes cost...,multiple: What does Mommy do as a job?,multiple: What was special about the boot?,one: Where does Mommy work?,multiple: Where did the needle the narrator st...,"{'A': 'filming movies', 'B': 'making costumes ...","{'A': 'It was covered in beads', 'B': 'it was ...","{'A': 'in the attic', 'B': 'in a costume shop'...","{'A': 'It was next to the lamp', 'B': 'It was ..."


In [4]:
test_stories = load_stories('data/mc500.test.tsv')
test_stories.T.head(2)

,story_id,story,q1,q2,q3,q4,q1_a,q2_a,q3_a,q4_a
0,mc500.test.0,It was Sally's birthday. She was very excited....,one: What time did the party start?,multiple: Who got hurt at the party?,one: Whose birthday is it?,multiple: What time did Jennifer arrive to the...,"{'A': '10', 'B': '2', 'C': '11', 'D': '1'}","{'A': 'Erin and Jennifer', 'B': 'Cathy and Eri...","{'A': 'Cathy', 'B': 'Jessica', 'C': 'Sally', '...","{'A': '1', 'B': '2', 'C': '8', 'D': '10'}"
1,mc500.test.1,On the farm there was a little piggy named And...,multiple: What did the piggies do when Andy go...,multiple: What did Andy see on his walk?,one: What did Andy love to roll around in?,one: What did the other piggies not want to do...,"{'A': 'play games and eat dinner', 'B': 'play ...","{'A': 'other piggies', 'B': 'flowers and a tre...","{'A': 'mud', 'B': 'cupcakes', 'C': 'grass', 'D...","{'A': 'walk', 'B': 'swim', 'C': 'eat', 'D': 'p..."


---
## Baselines

### Top Hypothesis Match (THM)

In [5]:
thm_baseline = THM_classifier(train_stories)
train_preds = thm_baseline.predict()
print(f"Baseline Accuracy on Training Data: {get_accuracy(train_preds, train_answers)}")

Baseline Accuracy on Training Data: 0.42083333333333334


In [6]:
weights = [1,1,1,1,1]
for ws in range(5):
    thm_baseline = THM_classifier(dev_stories, weights)
    print(f"Baseline Accuracy on Development Data with Weights = {weights}: {get_accuracy(thm_baseline.predict(), dev_answers)}")
    weights[1] += 1
    weights[2] += 6
    weights[3] += 1

Baseline Accuracy on Development Data with Weights = [1, 1, 1, 1, 1]: 0.46
Baseline Accuracy on Development Data with Weights = [1, 2, 7, 2, 1]: 0.485
Baseline Accuracy on Development Data with Weights = [1, 3, 13, 3, 1]: 0.475
Baseline Accuracy on Development Data with Weights = [1, 4, 19, 4, 1]: 0.475
Baseline Accuracy on Development Data with Weights = [1, 5, 25, 5, 1]: 0.475


In [7]:
thm_baseline_test = THM_classifier(test_stories, weights = [1,2,5,2,1])
baseline_test_preds = thm_baseline_test.predict()
baseline_test_preds['story_id'] = test_stories.T['story_id']
baseline_test_preds = baseline_test_preds.melt(id_vars = ['story_id'])
baseline_test_preds['variable'] += 1
baseline_test_preds['story_id'] = baseline_test_preds['story_id'] + '.' + baseline_test_preds['variable'].astype(str)
baseline_test_preds['sort'] = baseline_test_preds['story_id'].str.extract(r'\.(\d+)').astype(int)
baseline_test_preds = baseline_test_preds.sort_values('sort', kind = 'stable').drop(columns=['sort','variable']).rename(columns={'story_id' : 'id', 'value' : 'answer'})
print(baseline_test_preds.head())

#temp.to_csv('./test_answers.csv', sep = ',', header = True, index = False)

                 id answer
0    mc500.test.0.1      D
150  mc500.test.0.2      B
300  mc500.test.0.3      C
450  mc500.test.0.4      C
1    mc500.test.1.1      C


### Guess All C

In [8]:
get_accuracy(train_preds.replace(['A','B','D'],'C'), train_answers)

0.2525

In [9]:
get_accuracy(train_preds.replace(['A','B','D'],'C').iloc[:50], dev_answers)

0.275

---
## Entailment Model

In [10]:
train_statements = load_stories('./data/Statements/mc500.train.statements.tsv')
dev_statements = load_stories('./data/Statements/mc500.dev.statements.tsv')
test_statements = load_stories('./data/Statements/mc500.test.statements.tsv')

In [11]:
rte_model = RTE_classifier(train_answers)
train_df = rte_model.create_features(train_statements)
train_df.head()

,x1,x2,x3,x4,x5,x6,y
0,-0.590093,-0.376703,-1.350949,-0.240255,0.153940,-0.048334,0
1,-0.171678,-0.376703,-1.139616,-0.240255,-0.033792,0.369405,1
2,-0.171678,-0.376703,-0.992221,-0.240255,0.153940,0.156273,0
3,-0.590093,-0.376703,-1.079965,-0.240255,-0.033792,0.162193,0
4,-0.365656,-0.019366,-0.715916,-0.240255,-0.033792,-0.464155,0


In [12]:
rte_model.train(train_df)
train_preds = rte_model.predict(train_df.drop(columns = 'y'))
print(f"RTE Training Accuracy on MC500: {get_accuracy(train_preds['best'].values.reshape(-4,4), train_answers)}")

{'alpha': 0.01, 'eta0': 0.1, 'l1_ratio': 0.01, 'learning_rate': 'adaptive', 'loss': 'squared_hinge', 'max_iter': 10000, 'penalty': 'elasticnet'}
RTE Training Accuracy on MC500: 0.475


In [13]:
weights = [1,1,1,1,1]
for ws in range(5):
    rte_model.lambdas = weights
    dev_df = rte_model.create_features(dev_statements)
    dev_preds = rte_model.predict(dev_df.drop(columns = 'y'))
    print(f"RTE Training Accuracy on Dev Set with Weights = {weights}: {get_accuracy(dev_preds['best'].values.reshape(-4,4), dev_answers)}")
    weights[1] += 1
    weights[2] += 6
    weights[3] += 1

RTE Training Accuracy on Dev Set with Weights = [1, 1, 1, 1, 1]: 0.5
RTE Training Accuracy on Dev Set with Weights = [1, 2, 7, 2, 1]: 0.565
RTE Training Accuracy on Dev Set with Weights = [1, 3, 13, 3, 1]: 0.565
RTE Training Accuracy on Dev Set with Weights = [1, 4, 19, 4, 1]: 0.575
RTE Training Accuracy on Dev Set with Weights = [1, 5, 25, 5, 1]: 0.58


In [14]:
rte_model.lambdas =  [1,5,25,5,1]

In [15]:
test_df = rte_model.create_features(test_statements).drop(columns = 'y')
test_preds = rte_model.predict(test_df)
test_preds.head()

,A,B,C,D,best
0,0.243517,0.244315,0.243750,0.268418,D
1,0.252504,0.251554,0.247324,0.248619,A
2,0.242181,0.240963,0.276390,0.240466,C
3,0.260798,0.242483,0.254229,0.242490,A
4,0.235833,0.250272,0.242827,0.271068,D


In [16]:
pd.DataFrame(
    {
        'id' : baseline_test_preds['id'].values,
        'answer' : test_preds['best'].values
    }
).to_csv('./test_answers.csv', sep = ',', header = True, index = False)